In [1]:
import sys
import os

# Add the project root to the Python path
package_root = os.path.abspath(os.path.join(os.getcwd(), "../"))
print(f"Adding package root to sys.path: {package_root}")
if package_root not in sys.path:
	sys.path.append(package_root)


notebook_dir = os.getcwd()
print(f"Current notebook directory: {notebook_dir}")
# change to the directory to the root of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
print(f"Project root: {project_root}")
os.chdir(project_root)

Adding package root to sys.path: /home/mafzaal/source/lets-talk/py-src
Current notebook directory: /home/mafzaal/source/lets-talk/py-src/notebooks
Project root: /home/mafzaal/source/lets-talk


In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
import lets_talk.chains as chains
import lets_talk.prompts as prompts

In [40]:
# hot reload the module
import importlib
importlib.reload(chains)
importlib.reload(prompts)


<module 'lets_talk.prompts' from '/home/mafzaal/source/lets-talk/py-src/lets_talk/prompts.py'>

In [6]:
reponse = chains.tone_check_chain.invoke({"question": "I am so happy to be here!"})
reponse.content.lower() == "yes"

False

In [7]:
reponse = chains.tone_check_chain.invoke({"question": "Go to hell!"})
reponse.content.lower() == "yes"

True

In [18]:
from lets_talk.rag import rag_chain
reponse = rag_chain.invoke({"question": "Who is the data guy?"})

In [20]:
reponse["response"].content

"The Data Guy refers to Muhammad Afzaal, a data science expert and the author of the blog at [thedataguy.pro](https://thedataguy.pro). His work focuses on various topics in data science, AI evaluation, RAG systems, and metric-driven development, providing practical insights and frameworks for implementing these concepts effectively.\n\nIf you're interested in specific topics such as RAG systems, AI research agents, or data strategy, feel free to ask!"

In [13]:
from lets_talk.rag import rag_chain
reponse = rag_chain.invoke({"question": "What is speed of light!"})
reponse["response"].content

"I don't know the speed of light, but I can help you with topics related to data science, AI evaluation, RAG systems, and more. If you're interested in understanding how to evaluate AI agents or implement RAG systems, feel free to ask! You can also explore more on these topics at [TheDataGuy's blog](https://thedataguy.pro)."

In [8]:
from lets_talk.chains import rude_query_answer_chain
reponse = rude_query_answer_chain.invoke({"question": "Go to hell!"})

reponse.content

"I can sense that you're feeling really frustrated right now, and that's completely valid. We all have moments like that. 🌧️ \n\nIf there's something specific on your mind, I'm here to listen and help in any way I can. Let's turn this around and find a brighter perspective together! 🌈"

In [10]:
from lets_talk.chains import rude_query_answer_chain
reponse = rude_query_answer_chain.invoke({"question": "aweful!"})

reponse.content

"I understand that you might be feeling frustrated or disappointed, and that's completely valid. It's okay to express those feelings! Let's focus on finding something positive together. What’s something that brings you joy or makes you smile? 🌈"

In [ ]:
from lets_talk.chains import rude_query_answer_chain
reponse = rude_query_answer_chain.invoke({"question": "tell me a joke!"})

reponse.content

'**Input:** "Tell me a joke!"\n\n**Output:** "I love that you\'re looking for some humor! Laughter is such a wonderful way to brighten the day. Here’s a light-hearted joke for you: Why did the scarecrow win an award? Because he was outstanding in his field! 🌾😄 Keep smiling!"'

In [37]:
import lets_talk.agent as agent


In [79]:
importlib.reload(agent)


<module 'lets_talk.agent' from '/home/mafzaal/source/lets-talk/py-src/lets_talk/agent.py'>

In [80]:

uncompiled_graph = agent.build_graph()
graph =  uncompiled_graph.compile()

#show the graph


In [64]:
# from IPython.display import Image, display
# display(Image(graph.get_graph().draw_png()))

print(graph.get_graph().draw_ascii())

         +-----------+          
         | __start__ |          
         +-----------+          
                *               
                *               
                *               
    +---------------------+     
    | check_question_tone |     
    +---------------------+     
           .        ..          
         ..           .         
        .              ..       
+----------+             .      
| retrieve |           ..       
+----------+          .         
           *        ..          
            **    ..            
              *  .              
           +-------+            
           | agent |            
           +-------+            
           *        .           
         **          ..         
        *              .        
  +--------+       +---------+  
  | action |       | __end__ |  
  +--------+       +---------+  


In [81]:
graph_chain = agent.create_agent_chain(uncompiled_graph=uncompiled_graph)


In [74]:
response = graph_chain.invoke({"question": "tell me a joke!"})
response

{'messages': [HumanMessage(content='tell me a joke!', additional_kwargs={}, response_metadata={}, id='df2b46db-4109-448b-85a5-4fb91b0d1f36'),
  AIMessage(content="I don't know any jokes, but I can share some insightful content about data engineering or AI evaluation! If you're interested, let me know!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 990, 'total_tokens': 1020, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BWabqM5r06LNi5Ty3VRxWZwXYqIUk', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9874e8ab-653a-45f2-be45-0f3ce9de8ee7-0', usage_metadata={'input_tokens': 990, 'output_tokens': 30, 'total_tokens': 1020, 'input_token_details

In [49]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [82]:
response = graph_chain.invoke({"question": "Go to hell!"})
response

{'messages': [HumanMessage(content='Go to hell!', additional_kwargs={}, response_metadata={}, id='ada716be-d732-4df7-813f-8e4134bf86e6'),
  AIMessage(content="I can sense that you're feeling really frustrated right now, and that's completely valid. We all have moments like that. 🌧️ \n\nIf there's something specific on your mind, I'm here to listen and help in any way I can. Let's turn this around and find a brighter perspective together! 🌈", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 401, 'total_tokens': 462, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_129a36352a', 'id': 'chatcmpl-BWadnZqMM9ST3mPc8sigjYQNZQwdz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f

In [83]:
answer = agent.parse_output(response)
answer

"I can sense that you're feeling really frustrated right now, and that's completely valid. We all have moments like that. 🌧️ \n\nIf there's something specific on your mind, I'm here to listen and help in any way I can. Let's turn this around and find a brighter perspective together! 🌈"

In [76]:
response = graph_chain.invoke({"question": "Who are you?"})
response

{'messages': [HumanMessage(content='Who are you?', additional_kwargs={}, response_metadata={}, id='113cb4b2-040e-4bfd-801e-9fb32bade492'),
  AIMessage(content="I am TheDataGuy Chat, your specialized assistant for topics related to data science, AI evaluation, and metric-driven development, drawing insights from Muhammad Afzaal's blog at [thedataguy.pro](https://thedataguy.pro). My expertise includes:\n\n- RAG (Retrieval-Augmented Generation) systems and their implementation\n- Evaluation frameworks for AI applications\n- Building and assessing AI research agents\n- Data strategy and its significance for business success\n\nIf you have questions about these topics or need practical advice, feel free to ask! You can also explore more insights on the blog for in-depth articles and tutorials.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 988, 'total_tokens': 1116, 'completion_tokens_details': {'accepted_prediction_token

In [78]:
agent.parse_output(response)

"I am TheDataGuy Chat, your specialized assistant for topics related to data science, AI evaluation, and metric-driven development, drawing insights from Muhammad Afzaal's blog at [thedataguy.pro](https://thedataguy.pro). My expertise includes:\n\n- RAG (Retrieval-Augmented Generation) systems and their implementation\n- Evaluation frameworks for AI applications\n- Building and assessing AI research agents\n- Data strategy and its significance for business success\n\nIf you have questions about these topics or need practical advice, feel free to ask! You can also explore more insights on the blog for in-depth articles and tutorials."